In [1]:
%matplotlib inline

In [2]:
import gym
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import sys
import numpy as np
import math
import random 
import socket
import json
from socket import *
import copy
import struct
import time

class PositionControlEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        super(PositionControlEnv, self).__init__()

        # Define the low and high bounds for each action Kp, Kd, Ki
        low_bounds = np.array([0, 0, 0], dtype=np.float32)
        high_bounds = np.array([1, 1, 0.0001], dtype=np.float32)
        
        # Create the action space with custom bounds for each action
        self.action_space = spaces.Box(low=low_bounds, high=high_bounds, dtype=np.float32)

        # State: current position [-10, 10] + target position
        self.observation_space = spaces.Box(low=-1000, high=1000, shape=(3,), dtype=np.float32)
        self.max_step_per_episode = 512 
        self.flipped = False

        

    def reset(self):
        self.generate_random_positions()
        self.previous_error = np.array([0.0, 0.0, 0.0])
        self.sum_error = np.array([0.0, 0.0, 0.0])
        self.current_step = 0
        self.step_to_success = 0
        self.halfway = False
        self.flipped = False
        
        state = np.array([np.linalg.norm(self.position_error), np.linalg.norm(self.position_error-self.previous_error), np.linalg.norm(self.sum_error+self.position_error)])
        return state

    def generate_random_positions(self):
        # Generate random coordinates
        x = random.uniform(-1, 9)
        y = random.uniform(-1, 9)
        z = random.uniform(0.3, 3)
        self.target_position = np.array([0.0, 1.0, 0.7])
        self.current_position, _ = self.get_current_position()
        #self.target_position = np.array([x, y, z])
        
        self.position_error = self.target_position - self.current_position
        self.distance = np.linalg.norm(self.position_error)
        print("Current start_point: ", self.current_position)
        print("Current target: ", self.target_position)

            
    def step(self, action):            
        Kpid = action.reshape(-1)
        position_error = self.position_error
        #PID control logic 
        velocity = position_error * Kpid[0] + (position_error-self.previous_error)* Kpid[1] + (position_error + self.sum_error) * Kpid[2]
        normalized_v = np.array([x / (abs(x) + 1) for x in velocity]) # velocity ~ [-1,1]
        
        #print(Kpid)

        #FIXME: sending velocity 
        self.send_velocity(normalized_v[0], normalized_v[1],normalized_v[2])
        
        self.current_step += 1
        
        
        self.current_position, self.flipped = self.get_current_position()
        self.previous_error = position_error
        self.sum_error += position_error
        self.position_error = self.target_position - self.current_position # update position error after taking a step
        
        if self.flipped: #self.fail_threshold:
            reward = -1000 #self.fail_reward
            self.step_to_success = 0
            print("Drone tends to flip upside down, manual restart")
            done = True
            
        if not (-1 <= self.current_position[0] < 3) or not (-1 <= self.current_position[1] < 3) or self.current_position[2] > 2:
            reward = 0 #self.fail_reward
            self.step_to_success = 0
            print("Drone flies too far, manual restart")
            done = True
        elif np.linalg.norm(self.position_error) <= 0.25: #self.success_threshold:
            self.step_to_success += 1
            done = False
            reward = 0
            if self.step_to_success == 50:
                average_speed  = self.distance / ((self.current_step-50)*0.04) 
                reward = math.exp(average_speed*10)
                print("Reach the goal! Average flying speed:", average_speed, "m/s")
                #done = True
            
        elif np.linalg.norm(self.position_error) < (self.distance / 2): 
            reward = 0
            if self.halfway == False:
                halfway_speed  = self.distance / (self.current_step*0.04)
                reward = math.exp(halfway_speed*5)
                self.halfway = True
                print("Half way done! Keep going")
            self.step_to_success = 0
            done = False
        else:
            self.step_to_success = 0
            reward = 0
            done = False

        if self.current_step >= self.max_step_per_episode: #flexible self.max_steps_per_episode:
            print("Episode ends")
            reward = 0
            done = True
            self.send_velocity(0, 0, 0) #reset velocity when game over
        
        state = np.array([np.linalg.norm(self.position_error), np.linalg.norm(self.position_error-self.previous_error), np.linalg.norm(self.sum_error+self.position_error)])
        
        return state, reward, done, {}

    
    def close(self):
        if hasattr(self, 'figure'):
            plt.close(self.figure)


    def parseMessage(self, m):
	# Using dictionaries here is inefficient, but it makes things more readable
	# (i.e. change this aspect before implementing on a robot)
        packetInfo = { \
            'frame':0, \
            'numItems':0 \
            }
        objectData = { \
            'itemID':0, \
            'itemDataSize':0, \
            'objectName':'', \
            'xPos':0.0, \
            'yPos':0.0, \
            'zPos':0.0, \
            'xRot':0.0, \
            'yRot':0.0, \
            'zRot':0.0 \
            }
        byteIdx = 0

        packetInfo['frame'] = int.from_bytes(m[0][0:4], byteorder='little', signed=False)
        packetInfo['numItems'] = m[0][4]
        byteIdx = 5
        retObjects = []

        for numItems in range(0,packetInfo['numItems']):
            retObjects.append(copy.copy(objectData))

            retObjects[-1]['itemID'] = m[0][byteIdx]
            byteIdx = byteIdx + 1
		
		# **** NOTE: ****
		# This code assumes that each object has exactly 72 bytes of data, since it is
		# not documented where extra bytes would go if that field is set differently 
		# (although these extra/fewer bytes would probably go to/from the object name...)
		# ***************
            retObjects[-1]['itemDataSize'] = int.from_bytes(m[0][byteIdx:byteIdx+2], byteorder='little', signed=False)
            byteIdx = byteIdx + 2

            retObjects[-1]['objectName'] = m[0][byteIdx:byteIdx+24].decode('utf-8').strip(b'\x00'.decode())
            byteIdx = byteIdx + 24

            for floatParam in ['xPos','yPos','zPos','xRot','yRot','zRot']:
                retObjects[-1][floatParam] = struct.unpack('d',m[0][byteIdx:byteIdx+8])[0]/1000
                byteIdx = byteIdx + 8
                
        return packetInfo,retObjects


    #velocity controller    
    def get_current_position(self):
        try:
            # Open a UDP socket
            s=socket(AF_INET, SOCK_DGRAM)

            # UDP port 51001 is set by default in the vicon tracker software
            # ** Note: ** 
            #     This port number should be incremented by one on each robot if "object per port"
            #     is selected; the order of the objects is the order in which they appear in the
            #     tracker software
            udpPort = 65432

            # Bind the socket to the localhost on udpPort (which allows to read broadcast UDP packets)
            s.bind(('',udpPort))

            # 256 is set(able) in the vicon tracker software
            datagramSize = 256

            # Receive one packet:
            m = s.recvfrom(datagramSize)
            pi,ob = self.parseMessage(m)
            print('current position: ', [ob[0]['xPos'], ob[0]['yPos'], ob[0]['zPos']])
            return np.array([ob[0]['xPos'], ob[0]['yPos'], ob[0]['zPos']]), False
        except Exception as e:
            print("Failed to get position data:", str(e))
            return np.array([0, 0, 0]), True
            
        
    def send_velocity(self, vx, vy, vz):
        try:
            host = '192.168.11.1'  # The server's hostname or IP address on the drone side
            port = 65000  # The port used by the server on the drone side
            command = {'vx': vx, 'vy': vy, 'vz': vz}  # Construct the velocity command

            with socket(AF_INET, SOCK_STREAM) as s:
                s.connect((host, port))
                s.sendall(json.dumps(command).encode('utf-8'))  # Send the command as JSON
                print("Velocity command sent:", command)
                time.sleep(0.04)#???
        except Exception as e:
            print("Failed to send velocity command:", str(e))
            



In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Usage
env = DummyVecEnv([lambda: PositionControlEnv()])
model = PPO("MlpPolicy", env, n_steps=512, verbose=1)


c:\users\junyang\appdata\local\programs\python\python38\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


In [4]:
model = PPO.load("./PID_DRL_model_15000_steps", env=env, n_steps=512)
eval_env = PositionControlEnv()
eval_env.max_step_per_episode = 1000
obs = eval_env.reset()
step_to_success = 0
for _ in range(10000): 
    action, _states = model.predict(obs, deterministic=True)
    print(action)
    obs, reward, done, info = eval_env.step(action)
    if done:
        print("Game is Over", info)
        break
print("Game is Over", info)
eval_env.close()


current position:  [0.3191413444507537, 0.1032768639908614, 0.17200095205012103]
Current start_point:  [0.31914134 0.10327686 0.17200095]
Current target:  [0.  1.  0.7]
[3.43686e-02 0.00000e+00 1.00000e-04]
Velocity command sent: {'vx': -0.01088066438966636, 'vy': 0.029982081787062062, 'vz': 0.017874090830062212}
current position:  [0.3191435926112958, 0.10326638780580144, 0.1719942821977197]
[1.3038622e-01 0.0000000e+00 9.9999997e-05]
Velocity command sent: {'vx': -0.040008375429328065, 'vy': 0.1048258349327041, 'vz': 0.06450278493334663}
current position:  [0.3191769969005721, 0.10325262613878548, 0.17199116121167535]
[1.7229888e-01 0.0000000e+00 9.9999997e-05]
Velocity command sent: {'vx': -0.052213182248039315, 'vy': 0.13403237894394793, 'vz': 0.08352205545662403}
current position:  [0.3191704327216769, 0.1032608598883994, 0.17199324009564773]
[2.0291743e-01 0.0000000e+00 9.9999997e-05]
Velocity command sent: {'vx': -0.06093843407580359, 'vy': 0.15420721789070677, 'vz': 0.096945578

In [ ]:
eval_env = PositionControlEnv()
eval_env.get_current_position()
while True:
    eval_env.get_current_position()
    #eval_env.send_velocity(1, 2, 3)
#eval_env.send_velocity(2, 2, 2)